# SVM Manual Implementation

This is a one vs one manual implementation of the SVM algorithm. Most of the code is adapted from Matheiu Blondel's GitHub https://gist.github.com/mblondel/586753 which uses kernel implementations such as linear kernel, polynomial kernel and Gaussian kernel.

## Import Libraries

In [1]:
# Import libraries
import numpy as np
import cvxopt
import cvxopt.solvers
from scipy.stats import mode 
import cupy as cp
from matplotlib import pyplot as plt
from itertools import combinations
import seaborn as sns
import pandas as pd
import time
import json
import os
import scipy.io

In [2]:
# The folder name can be modified to whatever you wish
results_dir= "./slcw2_results/"

if not os.path.exists(results_dir):
    os.mkdir(results_dir)

## Helper Functions

### Kernel Functions

In [6]:
# Gaussian Kernel
def gaussian_kernel_matrix(X1, X2, c=1):
    
    """ 
    Compute gaussian kernel matrix, given data matrices.
    
    Args
    ----
    X1 - training data matrix of shape (m, n) where m is the number of training instances and n is the number of features
    X2 - matrix of shape (l, n) where l=m if we are generating matrix for training. Otherwise, l is the number of new points we are predicting on.
    c - width of Gaussian kernel
    
    Returns
    -------
    kernel matrix of dimensions (m, l)
    """
    
    B = X1 @ X2.T
    norm_sq = np.diagonal(X1@X1.T).reshape(-1, 1) - 2*B + np.diagonal(X2@X2.T).reshape(1, -1)
    return np.exp(-1 * c * norm_sq)

### Data Processing Functions

In [4]:
def preprocess(dataset, flat=True):
    """ Preprocess dataset by separating labels and image pixels. If flat=True, then each datapoint is a flattened vector. """

    # Extract labels
    y = dataset[:, 0]

    # Extract pixel values
    x = dataset[:, 1:]

    if not flat:
        # Reshape into image dimensions
        x = x.reshape((x.shape[0], 16, 16))

    return x, y

def create_subsets(data, labels, classes):
    
    """ 
    Create subsets of the dataset which only contain examples and labels for selected class pairs.
    
    Args
    ----
    data - dataset of dimensions (m, n) where m is the number of instances and n is the number of features
    labels - true labels for data, must be of shape (m, )
    classes - the class pairs to select from the full data matrix to generate data subset. Should be a tuple/list.
    
    Returns
    -------
    subset_examples - set of data examples corresponding to the desired class pairs
    subset_labels - true labels for the subset_examples.
    """
    
    # Find indices of rows with the desired classes
    class_1 = np.where(labels == classes[0])
    class_2 = np.where(labels == classes[1])
    
    # select rows from entire dataset
    subset_examples = np.concatenate((data[class_1], data[class_2]))
    subset_labels = np.concatenate((labels[class_1], labels[class_2]))
    
    return subset_examples, subset_labels

def split_data(inputs, targets, test_proportion, shuffle=None):
    """
    Splits the data into training and test sets.

    Args
    ----
    inputs : NumPy array of input data. Should be of shape (# examples, # features)
    targets : NumPy array of target data. Should be of shape (# examples, 1)
    test_proportion : Value between 0 and 1 which specifies how much of the data to use for testing.
    shuffle : Optional. Set to True if you want the data shuffled and then split.
    seed : Optional. Set for reproducible results.

    Returns
    -------
    train_X : NumPy array of training examples. Should be of shape (# examples, # features)
    train_Y : NumPy array of training targets. Should be of shape (# examples, 1)
    test_X : NumPy array of testing examples. Should be of shape (# examples, # features)
    test_Y : NumPy array of testing targets. Should be of shape (# examples, 1)
    """
  
    #Stores the number of data points
    nData = inputs.shape[0]

    # Shuffle data
    if shuffle:
        #Generate a shuffled version of the array indices
        shuffled_indices = np.random.permutation(nData)
        #Shuffle the inputs as per in the array of shuffled indices
        shuffled_inputs = inputs[shuffled_indices, :]
        shuffled_targets = targets[shuffled_indices]
    else:
        #If shuffle is set to False then we just work with the data in its original order
        shuffled_indices = None
        shuffled_inputs = inputs
        shuffled_targets = targets

    # Calculate the split index based on the specified proportions
    split_index = int((1 - test_proportion) * nData)
    
    # Collect train and test indices
    train_idxs = shuffled_indices[:split_index]
    test_idxs = shuffled_indices[split_index:]
    cache = (train_idxs, test_idxs)

    # Select the examples up to the split index to be used as training set
    train_X = shuffled_inputs[:split_index]
    train_Y = shuffled_targets[:split_index]
    # Select the examples from the split index onwards to be used as the test set
    test_X = shuffled_inputs[split_index:]
    test_Y = shuffled_targets[split_index:]

    return train_X, train_Y, test_X, test_Y, cache

## Load Full Dataset

In [5]:
# Load the full dataset - Save for later when everything works fine
full_dataset = np.genfromtxt("data/zipcombo.dat")

# Preprocess full dataset
x, y = preprocess(full_dataset)

# Inpsect the full dataset 
print("Number of records in full dataset = {}".format(full_dataset.shape[0]))
print("Labels for the full dataset = {}".format(np.unique(full_dataset[:, 0])))

Number of records in full dataset = 9298
Labels for the full dataset = [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


## Prepare SVM model

In [31]:
class multiclass_SVM():
    
    """ 
    
    Kernel Perceptron Class. Can be modified to train in one-vs-all (ova) mode or one-vs-one (ovo) mode.
    
    Args
    ----
    train_x : training data of shape (m, n) where m is the number of examples and n is number of features
    train_y : training labels of shape (m, )
    kernel : kernel matrix function

    Attributes
    ----------
    n_features : number of training features
    classes : list of unique classes in dataset
    classifiers : list of all pairwise combinations of classes
    train_x_subsets : list of subsets of data corresponding to class pairs
    train_y_subsets : list of labels for data subsets
    
    Methods
    -------
    fit - Fits all (n choose 2) classifiers with the SVM model defined in fit_single_classifier
    fit_single_classifier - performs SVM algorithm on single classifier
    predict - generates predictions on test points
    evaluate - computes misclassification error
    
    """
    
    def __init__(self, train_x, train_y, kernel):
        
        # Original datasets
        self.train_x = train_x
        self.train_y = train_y
        
        # kernel
        self.kernel = kernel
        
        # Dimensions
        self.n_features = train_x.shape[1]
        
        # Number of unique classes in dataset
        self.classes = np.unique(train_y)
        
        # creates all combinations of classes which will be used to design indivitual binary classifiers
        self.classifiers = list(combinations(self.classes, 2))

        # create data subsets 
        self.train_x_subsets = []
        self.train_y_subsets = []

        for i, classifier in enumerate(self.classifiers):

            # Extract class values
            class_1 = classifier[0]
            class_2 = classifier[1]

            # Create subset
            train_x_subset, train_y_subset = create_subsets(train_x, train_y, classifier)
            
            # Save train_x_subset
            self.train_x_subsets.append(train_x_subset)
            
            # Modify train_y_subset to have {-1, +1} labels
            train_y_subset = np.where(train_y_subset == class_1, 1., -1.)
            self.train_y_subsets.append(train_y_subset)
            
        # biases
        self.bs = []
        
        # lagrange multipliers and support vectors
        self.alphas = []
        self.svs = []
        
    def fit(self, C=None, verbose=True):
        
        """ Fits all (n choose 2) classifiers with the SVM model defined in fit_single_classifier """
        
        cvxopt.solvers.options['show_progress'] = False
        
        for i, classifier in enumerate(self.classifiers):
            
            # Log end time
            s = time.time()
            
            if verbose:
                print(">>>> Training Classifier {} : {} vs {}".format(i+1, classifier[0], classifier[1]), end="...")
            
            # obtain gram matrix and labels for corresponding classifier
            X = self.train_x_subsets[i]
            y = self.train_y_subsets[i]

            # Compute weights for the corresponding classifier
            self.fit_single_classifier(X, y, C)
            
            # Log start time
            e = time.time()
            
            if verbose:
                print("time taken: {:.5f} s".format(e - s))
            
    def fit_single_classifier(self, X, y, C):
        
        """ Runs SVM algorithm on single classifier using training data and C parameter """
        
        # Dimensions
        n_samples, n_features = X.shape
        
        # Compute kernel matrix
        K = self.kernel(cp.asarray(X), cp.asarray(X)).get()
        
        # initialize variables needed to solve QP problem
        P = cvxopt.matrix(np.outer(y,y)*K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        b = cvxopt.matrix(0.0)
        
        if C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples)*-1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples)*-1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        # obtain lagrange multipliers
        a = np.ravel(solution['x'])
        
        # support vector are those that have non zero lagrange multipliers
        sv = a > 1e-5
        
        # lagrange multipliers for sv points
        self.a = a[sv]
        
        # x and y values of sv points
        self.sv_x = X[sv]
        self.sv_y = y[sv] 
        
        # compute weights vector
        wx = (cp.asarray(self.a) * cp.asarray(self.sv_y)).reshape(1, -1) @ self.kernel(cp.asarray(self.sv_x), cp.asarray(self.sv_x))
        
        # compute intercept
        b = cp.mean(cp.asarray(self.sv_y) - wx).get()
        
        # save the b
        self.bs.append(b)
        self.alphas.append(self.a)
        self.svs.append((self.sv_x, self.sv_y))
    
    def predict(self, X):
        
        """ Generates predictions on new points """
        
        # Stores the classes predicted by the k(k-1)/2 indivitual binary classifiers in each row.
        # cols correspond to number of test examples
        class_matrix = np.zeros((len(self.classifiers), X.shape[0]))
        
        for i, classifier in enumerate(self.classifiers):
            
            # Extract class information
            class_1 = classifier[0]
            class_2 = classifier[1]
            
            # Generate predictions
            sv_x, sv_y = self.svs[i]
            wx = (cp.asarray(self.alphas[i]) * cp.asarray(sv_y)).reshape(1, -1) @ self.kernel(cp.asarray(sv_x), cp.asarray(X)) 
            b = cp.asarray(self.bs[i])
            pred = wx + b 
            
            # Encode as +1 or -1
            pred = np.where(cp.asnumpy(pred) > 0, 1, -1)
            
            # Convert back to the original k class values
            pred = np.where(pred == 1, class_1, class_2)
            
            # Update class matrix
            class_matrix[i, :] = pred
            
        # Convert to a final predictions vector where each element is the class with maximum vote
        class_predictions = mode(class_matrix)[0].squeeze()
        
        return class_predictions
    
    def evaluate(self, X, labels):
        
        """ Computes misclassification error, given data and true labels """
        
        # Generate predictions
        preds = self.predict(X)
        
        # Check for mistakes
        mistakes = np.where(preds != labels, 1.0, 0.0)
        
        return np.mean(mistakes)

# Run model for different polynomial values for 20 runs

In [14]:
def perform_multiple_runs(runs,
                          kernel_func,
                          param_values,
                          C_values,
                          save_results=False,
                          path_to_results="",
                          seed=None,
                          verbose=True):
    
    """
    Performs multiple runs of training with different parameter values.
    
    Args
    ----
    runs : number of runs to perform
    kernel_func : kernel matrix function
    param_values : parameters of kernel to train with
    C_values : list of C parameter values to train with
    save_results : Set true to save the results
    path_to_results : Saves results to provided filepath
    seed : to ensure reproducibility of results
    verbose : prints outputs while training
    
    Returns
    -------
    results_df : pandas dataframe of the final results from experiment.
    """

    # Save results here
    results = []

    # Set a random seed for reproducibility of results
    np.random.seed(seed)
    
    for C in C_values:

        for p in param_values:

            if verbose:
                print("paramter = {}, C = {}".format(p, C))
                
            result = {"C":C, "p":p, "Train Error":0, "Train Error STD":0, "Test Error":0, "Test Error STD":0}

            # Save errors for all runs here
            train_errors = []
            test_errors = []

            for run in range(runs):

                if verbose:
                    print(">> Run {}".format(run+1))

                # generate random train-test split
                train_x, train_y, test_x, test_y, _ = split_data(inputs=x, targets=y, test_proportion=0.20, shuffle=True)

                # setup kernel
                kernel = lambda X1, X2, eval: kernel_func(X1, X2, p)
                
                # Train svm model
                model = multiclass_SVM(train_x, train_y, kernel)
                model.fit(C=C, verbose=False)

                # evaluate on train and test set
                train_error = model.evaluate(train_x, train_y)
                test_error = model.evaluate(test_x, test_y)
                train_errors.append(train_error)
                test_errors.append(test_error)

            # Convert to numpy arrays
            train_errors = np.array(train_errors)
            test_errors = np.array(test_errors)
            
            if verbose:
                print("Mean Train Error = {}".format(np.mean(train_errors)))
                print("Mean Test Error = {}\n".format(np.mean(test_errors)))

            # Save results
            result["Train Error"] = np.mean(train_errors)
            result["Train Error STD"] = np.std(train_errors)
            result["Test Error"] = np.mean(test_errors)
            result["Test Error STD"] = np.std(test_errors)
            results.append(result)

    # convert matrix to a pandas dataframe for easier visualization
    results_df = pd.DataFrame(results, columns=[key for key in results[0].keys()])

    # Save results
    if save_results:
        results_df.to_csv(results_dir+path_to_results, header=True, index=True)

    return results_df

In [16]:
# parameters to run model on 
c_values = [2.0**-6]
C_values = [10**-1, 10**0, 10**1, 10**2, 10**3]
kernel = gaussian_kernel_matrix

# Run for 20 runs
results = perform_multiple_runs(runs=20,
                                param_values=c_values,
                                kernel_func=kernel,
                                C_values=C_values,
                                save_results=True,
                                path_to_results="q7_svm_basic_results_v2.csv",
                                seed=10)

paramter = 0.015625, C = 0.1
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> Run 9
>> Run 10
>> Run 11
>> Run 12
>> Run 13
>> Run 14
>> Run 15
>> Run 16
>> Run 17
>> Run 18
>> Run 19
>> Run 20
Mean Train Error = 0.04335170744823878
Mean Test Error = 0.059811827956989236

paramter = 0.015625, C = 1
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> Run 9
>> Run 10
>> Run 11
>> Run 12
>> Run 13
>> Run 14
>> Run 15
>> Run 16
>> Run 17
>> Run 18
>> Run 19
>> Run 20
Mean Train Error = 0.0015125033611185805
Mean Test Error = 0.02556451612903226

paramter = 0.015625, C = 10
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> Run 9
>> Run 10
>> Run 11
>> Run 12
>> Run 13
>> Run 14
>> Run 15
>> Run 16
>> Run 17
>> Run 18
>> Run 19
>> Run 20
Mean Train Error = 0.0002352783006184458
Mean Test Error = 0.02252688172043011

paramter = 0.015625, C = 100
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> 

In [55]:
# Check results
results = pd.read_csv(results_dir+"q7_svm_basic_results_v2.csv")
results

,Unnamed: 0,C,p,Train Error,Train Error STD,Test Error,Test Error STD
0,0,0.1,0.015625,0.043352,0.000982,0.059812,0.003910
1,1,1.0,0.015625,0.001513,0.000285,0.025565,0.003942
2,2,10.0,0.015625,0.000235,0.000058,0.022527,0.003480
3,3,100.0,0.015625,0.000094,0.000062,0.024194,0.002343
4,4,1000.0,0.015625,0.000000,0.000000,0.022419,0.002607


In [ ]:
plt.figure()
plt.plot(results['C'], results['Train Error'])
plt.plot(results['C'], results['Test Error'])
plt.show()

## Comparing with scikit's implementation (just to check if similar results are obtained)

In [33]:
from sklearn import svm

def perform_multiple_runs(runs,
                          kernel_func,
                          param_values,
                          C_values,
                          seed=None,
                          verbose=True):

    # Save results here
    results = []

    # Set a random seed for reproducibility of results
    np.random.seed(seed)
    
    for C in C_values:

        for p in param_values:

            if verbose:
                print("paramter = {}, C = {}".format(p, C))
                
            result = {"C":C, "p":p, "Train Error":0, "Train Error STD":0, "Test Error":0, "Test Error STD":0}

            # Save errors for all runs here
            train_errors = []
            test_errors = []

            for run in range(runs):

                if verbose:
                    print(">> Run {}".format(run+1))

                # generate random train-test split
                train_x, train_y, test_x, test_y, _ = split_data(inputs=x, targets=y, test_proportion=0.20, shuffle=True)
                
                # Train svm model
                model = svm.SVC(C=C, kernel=kernel_func, gamma=p, decision_function_shape="ovo")
                model.fit(train_x, train_y)
                
                # Generate predictions on train and test set
                train_preds = model.predict(train_x)
                test_preds = model.predict(test_x)

                train_mistakes = np.where(train_preds != train_y, 1.0, 0.0)
                test_mistakes = np.where(test_preds != test_y, 1.0, 0.0)
                
                # evaluate on train and test set
                train_error = np.mean(train_mistakes)
                test_error = np.mean(test_mistakes)
                train_errors.append(train_error)
                test_errors.append(test_error)

            # Convert to numpy arrays
            train_errors = np.array(train_errors)
            test_errors = np.array(test_errors)
            
            if verbose:
                print("Mean Train Error = {}".format(np.mean(train_errors)))
                print("Mean Test Error = {}\n".format(np.mean(test_errors)))

            # Save results
            result["Train Error"] = np.mean(train_errors)
            result["Train Error STD"] = np.std(train_errors)
            result["Test Error"] = np.mean(test_errors)
            result["Test Error STD"] = np.std(test_errors)
            results.append(result)

    # convert matrix to a pandas dataframe for easier visualization
    results_df = pd.DataFrame(results, columns=[key for key in results[0].keys()])
    
    return results_df

In [34]:
# parameters to run model on 
c_values = [2.0**-6]
C_values = [10**-1, 10**0, 10**1, 10**2, 10**3]

# Run for 20 runs
results_2 = perform_multiple_runs(runs=20,
                                param_values=c_values,
                                kernel_func="rbf",
                                C_values=C_values,
                                seed=10)

paramter = 0.015625, C = 0.1
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> Run 9
>> Run 10
>> Run 11
>> Run 12
>> Run 13
>> Run 14
>> Run 15
>> Run 16
>> Run 17
>> Run 18
>> Run 19
>> Run 20
Mean Train Error = 0.050813390696423766
Mean Test Error = 0.06728494623655915

paramter = 0.015625, C = 1
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> Run 9
>> Run 10
>> Run 11
>> Run 12
>> Run 13
>> Run 14
>> Run 15
>> Run 16
>> Run 17
>> Run 18
>> Run 19
>> Run 20
Mean Train Error = 0.0015192255982791072
Mean Test Error = 0.02545698924731183

paramter = 0.015625, C = 10
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> Run 9
>> Run 10
>> Run 11
>> Run 12
>> Run 13
>> Run 14
>> Run 15
>> Run 16
>> Run 17
>> Run 18
>> Run 19
>> Run 20
Mean Train Error = 0.0002218338262973918
Mean Test Error = 0.02228494623655914

paramter = 0.015625, C = 100
>> Run 1
>> Run 2
>> Run 3
>> Run 4
>> Run 5
>> Run 6
>> Run 7
>> Run 8
>> 

In [20]:
results

,Unnamed: 0,C,p,Train Error,Train Error STD,Test Error,Test Error STD
0,0,0.1,0.015625,0.043352,0.000982,0.059812,0.003910
1,1,1.0,0.015625,0.001513,0.000285,0.025565,0.003942
2,2,10.0,0.015625,0.000235,0.000058,0.022527,0.003480
3,3,100.0,0.015625,0.000094,0.000062,0.024194,0.002343
4,4,1000.0,0.015625,0.000000,0.000000,0.022419,0.002607


In [35]:
results_2

,C,p,Train Error,Train Error STD,Test Error,Test Error STD
0,0.1,0.015625,0.050813,0.001241,0.067285,0.004006
1,1.0,0.015625,0.001519,0.000255,0.025457,0.003494
2,10.0,0.015625,0.000222,0.000064,0.022285,0.003407
3,100.0,0.015625,0.000094,0.000062,0.024382,0.002580
4,1000.0,0.015625,0.000000,0.000000,0.022392,0.003038


# Run 20 runs of 5 fold Cross validation

In [36]:
def perform_kfoldCV(k, x, y, C, p, kernel_func, shuffle=True, verbose=True):
    
    """ 
    Performs k-fold cross validation for given parameters
    
    Args
    ----
    k - number of folds of CV to perform
    x - training data
    y - training labels
    C - C parameter for SVM
    p - parameter to use for kernel
    kernel_func - kernel matrix function
    shuffle - shuffles data before performing k-fold CV
    verbose - prints output logs while running
    
    Returns
    -------
    cv_error_over_folds - the cross valiation error for each fold of cross validation.
    """

    # Extract dimensions
    m, n = x.shape
    
    # Configure kernel function to have a specific d value  
    kernel = lambda X1, X2: kernel_func(X1, X2, p)

    # Shuffle dataset randomly for splitting into groups
    if shuffle:
        perm = np.random.permutation(m)
        x_shuffled = x[perm, :]
        y_shuffled = y[perm]
    else:
        x_shuffled = x
        y_shuffled = y

    # Split data into k-groups
    x_groups = np.array_split(x_shuffled, k)
    y_groups = np.array_split(y_shuffled, k)

    # Stores the mean CV error over all folds of CV
    cv_error_over_folds = 0

    for i in range(len(x_groups)):
        
        if verbose:
            print(">>>> Cross-validation Fold {}".format(i+1), end="...")

        # Use the selected group as "validation" set
        val_inputs, val_labels = x_groups[i], y_groups[i]

        # Use rest of groups as training set
        train_inputs = np.vstack([x_groups[j] for j in range(len(x_groups)) if j != i])
        train_labels = np.concatenate([y_groups[j] for j in range(len(x_groups)) if j != i])

        #-------------------------TRAIN MODEL--------------------------#

        # Train model
        model = multiclass_SVM(train_inputs, train_labels, kernel)
        model.fit(C=C, verbose=False)
        cv_error_over_folds += model.evaluate(val_inputs, val_labels)

        if verbose:
            print("Done!")
            
        #--------------------------------------------------------------#
            
    #Average the errors
    cv_error_over_folds /= k

    return cv_error_over_folds

In [37]:
def perform_multiple_runs_with_kFold(runs,
                                     param_values,
                                     kernel_func,
                                     C_values,
                                     k=5,
                                     seed=None,
                                     verbose=True,
                                     save_results=True,
                                     path_to_results=""):
    
    """
    Performs multiple runs of training with different parameter values.
    
    Args
    ----
    param_values : list of parameter values to train on
    k : number of folds of CV
    C_values : list of C parameters to train with
    kernel_func : kernel matrix function
    runs : number of runs to perform
    seed : to ensure reproducibility of results
    verbose : prints outputs while training
    save_results : Set true to save the results
    path_to_results : Saves results to provided filepath
    
    Returns
    -------
    results_df : pandas dataframe of the final results from experiment.
    """

    # Set a random seed for reproducibility of results
    np.random.seed(seed)

    # Results will be stored here
    results = np.zeros((20, 3))

    for run in range(runs):
        
        if verbose:
            print("Run {}".format(run+1))

        # generate random train-test split
        train_x, train_y, test_x, test_y, _ = split_data(inputs=x, targets=y, test_proportion=0.20, shuffle=True)

        #----------------------------PERFORM k-FOLD CV-------------------------------#
        
        # I will record the errors for a single run in this vector
        grid_search = []
    
        for C in C_values:

            for p in param_values:

                if verbose:
                    print(">> C = {}, parameter = {}".format(C, p))
                    
                # Perform k-fold CV on the training set
                mean_cv_error = perform_kfoldCV(k, train_x, train_y, C, p, kernel_func, verbose=verbose)
                
                result = {"C":C, "p":p, "CV_Error":mean_cv_error}
                
                grid_search.append(result)
                
        #-----FIND BEST HYPERPARAM VALUE AND TRAIN THE WHOLE DATASET WITH THAT------#

        idx_least_error = np.argmin([model["CV_Error"] for model in grid_search])
        best_p = grid_search[idx_least_error]["p"]
        best_C = grid_search[idx_least_error]["C"]
        
        if verbose:
            print("\nBest model params : p = {}, C = {}".format(best_p, best_C), end="...")

        # Set the kernel to have this d*
        kernel = lambda X1, X2: kernel_func(X1, X2, best_p)
        
        if verbose:
            print("Training best model", end="...")

        # Fit model on training set
        model = multiclass_SVM(train_x, train_y, kernel)
        model.fit(C=best_C, verbose=False)

        # Evaluate on test set
        test_error = model.evaluate(test_x, test_y)
        
        if verbose:
            print("Test Error = {}\n".format(test_error))

        # Record results
        results[run, 0] = best_p
        results[run, 1] = best_C
        results[run, 2] = test_error

    if save_results:
        # Convert results matrix to pandas dataframe
        results_df = pd.DataFrame(results, columns=["best p", "best_C", "Test Error"], index=["Run {}".format(run+1) for run in range(runs)])
        results_df.to_csv(results_dir+path_to_results, header=True, index=True)
        
    return results_df

In [38]:
# Range of d values
p_values = [2.0**-6]
C_values = [10**-1, 10**0, 10**1, 10**2, 10**3]
kernel = gaussian_kernel_matrix

# Run model for multiple runs with cross validation
results_df = perform_multiple_runs_with_kFold(runs=20,
                                              param_values=p_values,
                                              kernel_func=kernel,
                                              C_values=C_values,
                                              k=5,
                                              seed=231,
                                              path_to_results="q7_manual_svm_crossval_v2.csv")

Run 1
>> C = 0.1, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation Fold 5...Done!
>> C = 1, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation Fold 5...Done!
>> C = 10, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation Fold 5...Done!
>> C = 100, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation Fold 5...Done!
>> C = 1000, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-valida

>>>> Cross-validation Fold 5...Done!
>> C = 1000, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation Fold 5...Done!

Best model params : p = 0.015625, C = 10...Training best model...Test Error = 0.01989247311827957

Run 15
>> C = 0.1, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation Fold 5...Done!
>> C = 1, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation Fold 5...Done!
>> C = 10, parameter = 0.015625
>>>> Cross-validation Fold 1...Done!
>>>> Cross-validation Fold 2...Done!
>>>> Cross-validation Fold 3...Done!
>>>> Cross-validation Fold 4...Done!
>>>> Cross-validation F

In [54]:
print("\nMean d* \u00B1 STD = {} \u00B1 {}".format(results_df["best_C"].mean(), results_df["best_C"].std()))
print("Mean Test Error \u00B1 STD = {} \u00B1 {}".format(results_df["Test Error"].mean(), results_df["Test Error"].std()))


Mean d* ± STD = 392.5 ± 458.7956201323543
Mean Test Error ± STD = 0.02298387096774194 ± 0.001995508584779877


# OLD CODE

In [ ]:
class SVM():
    
    """ Trains an SVM classifier using a simplified version of the SMO algorithm """
    
    def __init__(self, train_x, train_y, test_x, test_y, kernel_func):
        
        self.classes = np.unique(train_y)
        
        # creates all combinations of classes which will be used to design indivitual binary classifiers
        self.classifiers = list(combinations(self.classes, 2))

        # initialize datasets
        self.alphas = []
        self.bs = []
        self.classifier_K = []  
        self.classifier_y = [] 
        self.train_K = [] 
        self.train_y = train_y 
        self.test_K = [] 
        self.test_y = test_y 

        for i, classifier in enumerate(self.classifiers):

            # Extract class values
            class_1 = classifier[0]
            class_2 = classifier[1]

            # Create subset
            examples_subset, labels_subset = create_subsets(train_x, train_y, classifier)

            # Create sub-kernels for each classifier
            classifier_kernel = cp.asnumpy(kernel_func(cp.asarray(examples_subset), cp.asarray(examples_subset), eval=False))

            # Create train kernels
            train_kernel = cp.asnumpy(kernel_func(cp.asarray(examples_subset), cp.asarray(train_x), eval=True))

            # Create test kernel
            test_kernel = cp.asnumpy(kernel_func(cp.asarray(examples_subset), cp.asarray(test_x), eval=True))

            # Save kernels
            self.classifier_K.append(classifier_kernel)
            self.train_K.append(train_kernel)
            self.test_K.append(test_kernel)

            # Convert labels to {+1, -1} encoding.
            labels_subset = np.where(labels_subset == class_1, 1, -1)
            self.classifier_y.append(labels_subset)

            # Initialize weights as 0s
            self.alphas.append(np.zeros(len(examples_subset)))
            self.bs.append(0)
        
    def fit(self, C, tol=1e-3, max_passes=5):
        
        for i, classifier in enumerate(self.classifiers):
            
            s = time.time()
            
            print(">>>> Training Classifier {} : {} vs {}".format(i+1, classifier[0], classifier[1]), end="...")
            
            K = self.classifier_K[i]
            y = self.classifier_y[i]
            
            alphas, b = self.fit_single_classifier(K, y, C, tol, max_passes)
            
            self.alphas[i] = alphas
            self.bs[i] = b
            
            e = time.time()
            
            print("time taken: {:.5f} s".format(e - s))
            
    def fit_single_classifier(self, K, Y, C, tol, max_passes):
        
        """ 

        Args
        ----
        X : matrix of training examples where each row is an example and j-th column is j-th feature.
        Y : column vector containing 1 for positive examples and -1 for negative examples.
        C : regularization parameter for SVM
        kernel : function to compute kernel 
        tol : tolerance value for determining equality of floating point numbers.
        max_passes : controls number of iterations.
    
        """
        
        m = K.shape[0]

        # Variables
        alphas = np.zeros((m, 1))
        b = 0
        E = np.zeros((m, 1))
        passes = 0
        eta = 0
        L = 0
        H = 0

        while passes < max_passes:
            
            num_changed_alphas = 0

            for i in range(m):

                # Calculate Ei = f(x_i) - y_i using (2)
#                 E[i] = b + np.sum(alphas * Y * K[:, i]) - Y[i]
                E[i] = b + (cp.asarray(alphas).T @ (cp.asarray(K[:, i]) * cp.eye(m)) @ cp.asarray(Y)).get() - Y[i]

                if ((Y[i]*E[i] < -tol and alphas[i] < C) or (Y[i]*E[i] > tol and alphas[i] > 0)):

                    # Select j != i randomly
                    j = np.random.randint(0, m)
                    while j == i:
                        j = np.random.randint(0, m)

                    # Calculate Ej = f(x(j)) - y(j) using (2)
#                     E[j] = b + np.sum(alphas * Y * K[:, j]) - Y[j]
                    E[j] = b + (cp.asarray(alphas).T @ (cp.asarray(K[:, j]) * cp.eye(m)) @ cp.asarray(Y)).get() - Y[j]

                    # Save old alphas
                    alpha_i_old = alphas[i]
                    alpha_j_old = alphas[j]

                    # Compute L and H by (10) or (11)
                    if (Y[i] == Y[j]):
                        L = max(0, alphas[j] + alphas[i] - C)
                        H = min(C, alphas[j] + alphas[i])
                    else:
                        L = max(0, alphas[j] - alphas[i])
                        H = min(C, C + alphas[j] - alphas[i])

                    if (L == H):
                        # continue to next i
                        continue
                        
                    # Compute eta by (14)
                    eta = 2 * K[i, j] - K[i, i] - K[j, j]
                    
                    if (eta >= 0):
                        continue

                    # Compute and clip new value for alpha j using (12) and (15)
                    alphas[j] = alphas[j] - (Y[j] * (E[i] - E[j])) / eta

                    # Clip 
                    alphas[j] = min(H, alphas[j]) 
                    alphas[j] = min(L, alphas[j])

                    # Check if change in alpha is significant
                    if (np.abs(alphas[j] - alpha_j_old) < tol):
                        # continue to next i
                        # replace anyway
                        alphas[j] = alpha_j_old
                        continue

                    # Determine value for alpha i using (16)
                    alphas[i] = alphas[i] + Y[i]*Y[j]*(alpha_j_old - alphas[j])

                    # Compute b1 and b2 using (17) and (18) respectively
                    b1 = b - E[i] - Y[i] * (alphas[i] - alpha_i_old) * K[i, j].T - Y[j] * (alphas[j] - alpha_j_old) * K[i, j].T
                    b2 = b - E[j] - Y[i] * (alphas[i] - alpha_i_old) * K[i, j].T - Y[j] * (alphas[j] - alpha_j_old) * K[j, j].T

                    # Compute b by (19)
                    if (0 < alphas[i] and alphas[i] < C):
                        b = b1
                    elif (0 < alphas[j] and alphas[j] < C):
                        b = b2
                    else:
                        b = (b1 + b2)/2
                        
                    num_changed_alphas += 1

            if (num_changed_alphas == 0):
                passes += 1
            else:
                passes = 0
                
        return alphas, b
    
    def predict(self, K, labels):
        
        # Stores the classes predicted by the k(k-1)/2 indivitual binary classifiers in each row.
        # cols correspond to number of test examples
        class_matrix = np.zeros((len(self.classifiers), K[0].shape[1]))
        
        for i, classifier in enumerate(self.classifiers):
            
            # Extract class information
            class_1 = classifier[0]
            class_2 = classifier[1]
            
            # Generate predictions
            pred = cp.asarray(self.bs[i]) + (cp.asarray(self.alphas[i]).squeeze() * cp.asarray(labels[i]).T) @ cp.asarray(K[i])
            
            # Encode as +1 or -1
            pred = np.where(cp.asnumpy(pred) > 0, 1, -1)
            
            # Convert back to the original k class values
            pred = np.where(pred == 1, class_1, class_2)
            
            # Update class matrix
            class_matrix[i, :] = pred
            
        # Convert to a final predictions vector where each element is the class with maximum vote
        class_predictions = mode(class_matrix)[0].squeeze()
        
        return class_predictions
    
    def evaluate(self, K, labels):
        
        # Generate predictions
        preds = self.predict(K, labels)
        
        # Check for mistakes
        mistakes = np.where(preds != labels, 1.0, 0.0)
        
        return np.mean(mistakes)